In [1]:
from bs4 import BeautifulSoup
from datetime import date, datetime
from twilio.rest import Client
import requests
import re
import pandas as pd
import numpy as np


In [2]:
def _max(x):
    if type(x) == list:
        x_list = list(filter(None, x))
        return max(list(map(int, x_list))) / 100
    return float(x) / 100

def is_owner_operated(x):
    if type(x) == str:
        return x.find('OWNER') > -1

def separate_rent(total_rent):
    rent = [float(x)/100 for x in total_rent if x != '']
    base_rent = nnn = total = 0
    if len(rent) == 3:
        [base_rent, nnn, total] = rent
        return [base_rent, nnn, total]
    else:
        base_rent = rent[0]
        total = max(rent)
        return [base_rent, nnn, total]
    
def separate_employees(x):
    if type(x) == str:
        employeelist = re.findall('\d*\D+',x)
        full_time=0
        personal_trainer=0
        part_time=0
        unknown=0
        
        for item in employeelist:
            if item.find('FULL')>-1:
                full_time = item.split(' ')[0]
            elif item.find('TRAIN')>-1 or item.find('PT')>-1:
                personal_trainer = item.split(' ')[0]
            elif item.find('PART')>-1:
                part_time = item.split(' ')[0]
            elif item.find('OWN') >-1:
                owner=True
            else:
                unknown = item.split(' ')[0]
            
        return employeelist, full_time, part_time, personal_trainer, unknown

def separate_city_state(location):
    city = location[0]
    state = location[1]
    return [city, state]

def send_sms(listings_filename, number_of_listings):
    client = Client("AC6cc4d0938f56288b419b5c9678eed77b","c74f93f4eafed5a3c2ac688e81ea82c3")
    client.messages.create(to="+14093441539",
                          from_="+14092047228",
                          body="New Anytime Listings file downloaded: " + listings_filename + "\nNumber of Listings: " + str(number_of_listings))

def text_is_location(text):
    return re.search(r'^(?!.*:).*, [A-Z]{2}', text) != None

def add_listing_values_to_df(keys, values, df):
    dictionary = dict(zip(keys, values))
    dfTemp = pd.DataFrame(dictionary, index=[0])
    df = df.append(dfTemp, ignore_index=True)#, sort=False)
    return df

def text_is_listing_detail(text):
    return element.text.find(':')

def clean_key(key):
    key = key.strip().upper()
    if key.startswith("ENHANCEMENT") or key.startswith("CLUB REINVENTION"):
        key = "ENHANCEMENT DUE DATE"
    elif key.startswith("REASON"):
        key = "REASON FOR SELLING"
    return key

In [3]:
url = "https://content.anytimefitness.com/display/AFCR/Online+Club+Listings"
response = requests.get(url)
content = response.content
soup = BeautifulSoup(content, 'html.parser')
#listings = soup.find_all(id=re.compile("OnlineClubListings-[^.\"](?!(oreceive)|(LICKTO))"))
content = soup.find('div', id='content')


In [4]:
#soup.find_all(id=re.compile("OnlineClubListings-[^.\"](?!(oreceive)|(LICKTO))"))
listings = soup.find_all('li', class_='child-item')
dfAddresses = pd.DataFrame()
for listing in listings:
    address = listing.text.split(',')
    link = 'https://content.anytimefitness.com' + listing.a.get('href')
    if len(address) == 3:
        dfAddresses = dfAddresses.append({'STREET': address[0].strip(),
                                          'CITY': address[1].strip(),
                                          'STATE': address[2].strip(),
                                          'LINK': link}, ignore_index=True)

In [5]:
dfListings = pd.DataFrame()
keys=[]
columns=[]

for element in content.next_elements:     
    if element.name == 'h2' or element.name == 'p':
        if text_is_location(element.text):
            
            # Beginning of new entry
            # Append dfListing to dfListings
            if len(keys)>0: 
                dfListings = add_listing_values_to_df(keys, values, dfListings)
            
            # Add location to key/value
            location = element.text
            keys = ['DATE ACQUIRED', 'LOCATION', 'LINK_KEY']
            
            # Get hyperlink to use as key
            
            f = element.find('a', href=True)
            if f != None:
                link = f.get('href')
            else:
                e2 = element.next_sibling
                #while True:
                f = e2.find('a', href=True)
                if f != None:
                    link = f.get('href')
                else:
                    link = e2.get('href')
                    
                if link.find('maps')>-1:
                    while True:
                        e2 = e2.next_sibling
                        f = e2
                        if f.has_attr('href'):
                            link = f.get('href')
                            break
           
            values = [date.today(), location, link]
            
        elif text_is_listing_detail(element.text):
            columns = element.text.strip().split(":", 1) 
            if len(columns)==2:
                key, value = columns
                key = clean_key(key)
                keys.append(key)
                values.append([value.strip()])

# Add last entry to df
dfListings = add_listing_values_to_df(keys, values, dfListings)

In [6]:
location = dfListings['LOCATION'].str.split(', ', 1)
dfListings['CITY'] = location.str[0].str.strip()
state_street = location.str[1].str.strip().str.split(' ', 1)
dfListings['STATE'] = state_street.str[0].str.strip()
dfListings['STREET'] = state_street.str[1].str.strip()
rent = dfListings['TOTAL RENT'].str.replace(r'[,\.\$]', '').str.replace(r'\D', ' ').str.split(' ').apply(separate_rent)
dfListings['BASE RENT'] = pd.to_numeric(rent.str[0])
dfListings['NNN'] = pd.to_numeric(rent.str[1])
dfListings['TOTAL RENT VALUE'] = pd.to_numeric(rent.str[2])

dfListings['OWNER OPERATED'] = dfListings['NUMBER OF EMPLOYEES'].str.upper().apply(is_owner_operated)

employees = dfListings['NUMBER OF EMPLOYEES'].str.upper().apply(separate_employees)
dfListings['EMP FULL TIME'] = pd.to_numeric(employees.str[1])
dfListings['EMP PART TIME'] = pd.to_numeric(employees.str[2])
dfListings['EMP PERSONAL TRAINER'] = pd.to_numeric(employees.str[3])
dfListings['ASKING PRICE (NUMERIC)'] = pd.to_numeric(dfListings['PRICE'].str.split(' ').str[0].str.replace(r'\D',''))
dfListings['ACTIVE MEMBERS'] = pd.to_numeric(dfListings['ACTIVE MEMBERS'].str.replace(r'\D',''))
dfListings['PAY PER VISIT MEMBERS'] = pd.to_numeric(dfListings['PAY PER VISIT MEMBERS'].str.replace(r'\D',''))
dfListings['TOTAL MEMBERS'] = pd.to_numeric(dfListings['TOTAL MEMBERS'].str.replace(r'\D',''))
dfListings['SQUARE FOOTAGE'] = pd.to_numeric(dfListings['SQUARE FOOTAGE'].str.replace(r'\D',''))


In [7]:
dfListings['ACTIVE MEMBER COUNT THAT PAYS ANNUALY'] = 0
dfListings['TRANSFER FEE'] = 7500
dfListings['ACTIVE MEMBER RATE'] = 36.08
dfListings['ACTIVE MEMBER INCOME'] = dfListings['ACTIVE MEMBER RATE'] * dfListings['ACTIVE MEMBERS']
dfListings['PAY PER VISIT MEMBER RATE'] = 8.05
dfListings['PAY PER VISIT MEMBER INCOME'] = dfListings['PAY PER VISIT MEMBER RATE'] * dfListings['PAY PER VISIT MEMBERS']
dfListings['TOTAL INCOME FROM MONTHLY DUES'] = dfListings['ACTIVE MEMBER INCOME'] + dfListings['PAY PER VISIT MEMBER INCOME']
dfListings['FRANCHISE MONTHLY FEE'] = -699
dfListings['GENERAL ADVERTISING FEE'] = -600
dfListings['BASE TECHNOLOGY FEE'] = -799
dfListings['TRAINING SUITE FEE'] = 0
dfListings['ON-SITE RELAUNCH TRAINING FEES'] = 0
dfListings['MARKETING MATERIALS'] = 0
dfListings['INSURANCE'] = -225
dfListings['UTILITIES'] = -500
dfListings['MAINTENANCE'] = -150
dfListings['SUPPLIES'] = -75
dfListings['COST OF SALE???'] = -1000
dfListings['ADMINISTRATIVE FEE???'] = -600
dfListings['CLUB ENHANCEMENT SAVINGS'] = -500
dfListings['LOCAL MARKETING BUDGET'] = -1000
dfListings['TOTAL MONTHLY FEES'] = (dfListings['FRANCHISE MONTHLY FEE']
    + dfListings['GENERAL ADVERTISING FEE']
    + dfListings['BASE TECHNOLOGY FEE']
    + dfListings['TRAINING SUITE FEE']
    + dfListings['ON-SITE RELAUNCH TRAINING FEES']
    + dfListings['MARKETING MATERIALS']
    + dfListings['INSURANCE']
    + dfListings['CLUB ENHANCEMENT SAVINGS']
    + dfListings['LOCAL MARKETING BUDGET']
    + dfListings['UTILITIES']
    + dfListings['MAINTENANCE']
    + dfListings['SUPPLIES']
    + dfListings['COST OF SALE???']
    + dfListings['ADMINISTRATIVE FEE???'])
dfListings['FULL TIME ANNUAL SALARY'] = -31200
dfListings['PART TIME ANNUAL SALARY'] = -31200/2
dfListings['FULL TIME MONTHLY WAGES'] = -31200 * dfListings['EMP FULL TIME'] / 12
dfListings['PART TIME MONTHLY WAGES'] = dfListings['PART TIME ANNUAL SALARY'] * dfListings['EMP PART TIME'] / 12
dfListings['TOTAL MONTHLY WAGES'] = dfListings['FULL TIME MONTHLY WAGES'] + dfListings['PART TIME MONTHLY WAGES']
dfListings['TOTAL MONTHLY EXPENSES'] = dfListings['TOTAL MONTHLY WAGES'] + dfListings['TOTAL MONTHLY FEES'] + (-dfListings['TOTAL RENT VALUE'])
dfListings['INCOME AFTER MONTHLY EXPENSES'] = dfListings['TOTAL INCOME FROM MONTHLY DUES'] + dfListings['TOTAL MONTHLY EXPENSES']
dfListings['ANNUAL INCOME'] = dfListings['INCOME AFTER MONTHLY EXPENSES'] * 12
dfListings['PURCHASE PRICE (WITH TRANSFER FEE)'] = dfListings['ASKING PRICE (NUMERIC)'] + dfListings['TRANSFER FEE']
dfListings['ANNUAL ROI'] = dfListings['ANNUAL INCOME'] / dfListings['PURCHASE PRICE (WITH TRANSFER FEE)']

dfListings['DOES THE CLUB NEED ANY UPDATES REQUIRED BY CORPORATE?']=''
dfListings['MAKE SURE THE OWNER SIGNS A NON-COMPETE CLAUSE'] = ''
dfListings['GET ITEMIZED EQUIPMENT LIST WITH CURRENT VALUE AND REPLACEMENT VALUE'] = ''
dfListings['NEGOTIATE WHO WILL PAY THE TRANSFER FEE'] = ''
dfListings['HOW LONG HAS THE CURRENT OWNER OWNED THE GYM?'] = ''
dfListings['HOW LONG HAS THE CURRENT EMPLOYEES BEEN AT THE GYM?'] = ''
dfListings['HOW HAS MEMBERSHIP RATES CHANGED OVER THE LAST 2 OR 3 YEARS'] = ''
dfListings['CLUB OWNER NAME'] = ''
dfListings['CLUB OWNER EMAIL ADDRESS'] = ''
dfListings['FULL ADDRESS2'] = dfListings['LINK_KEY'].str.replace('/display/AFCR/','').str.replace('%2C',',').str.replace('+', ' ')

In [8]:
for listing in dfListings.itertuples():
    df_state_match = dfAddresses.loc[dfAddresses.STATE==listing.STATE]
    df_city_match = df_state_match.loc[df_state_match.CITY == listing.CITY]
    if df_city_match.shape[0] == 1:
        dfListings.at[listing.Index, 'STREET'] = df_city_match.STREET.iloc[0]
        dfListings.at[listing.Index, 'LINK'] = df_city_match.LINK.iloc[0]
        dfListings.at[listing.Index, 'FULL ADDRESS'] = df_city_match.STREET.iloc[0] + ', ' + listing.CITY + ', ' + listing.STATE
    else:
        for city in df_city_match.itertuples():
            if str(city.STREET).find(str(listing.STREET).split()[0].replace("(",""))>=0:
                dfListings.at[listing.Index, 'STREET'] = city.STREET
                dfListings.at[listing.Index, 'LINK'] = city.LINK
                dfListings.at[listing.Index, 'FULL ADDRESS'] = city.STREET + ', '+ listing.CITY + ', ' + listing.STATE
                break

In [9]:
# Open Existing data_file and compare to dataframe.
# If there are new records then add them.
# If there are updated pricing then add them.

dfRecord = pd.read_excel('./listings.xlsx', index_col=0)

In [31]:
#new_listings = ''
dfFile = dfRecord.reset_index(drop=True)
dfNew = dfListings.reset_index(drop=True)
#dfFile['ReducedPrice?'] = np.where(dfFile['LINK_KEY'] == dfNew['LINK_KEY'], dfNew['ASKING PRICE (NUMERIC)'] < dfFile['ASKING PRICE (NUMERIC)'],'0')
#sale_pending_listings = ''
#removed_listings = ''
np.where(dfFile['LINK_KEY'] == dfNew['LINK_KEY'], 'True', 'False')

ValueError: Can only compare identically-labeled Series objects

In [30]:
dfRecord.columns.values ==dfListings.columns.values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True], dtype=bool)

In [10]:
# New listings
df_all = dfListings.merge(dfRecord, on=['LINK_KEY'], how='left', indicator=True)
df_all['_merge'] == 'left_only'

# Reduced Price Listings


0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
      ...  
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
59    False
60    False
61    False
62    False
63    False
64    False
65    False
66    False
67    False
68    False
69    False
70    False
71    False
72    False
73    False
74    False
75    False
76    False
77    False
78    False
79    False
Name: _merge, Length: 80, dtype: bool

In [77]:
dfListings.head()

,ACTIVE MEMBERS,CLUB MANAGEMENT SYSTEM,CLUB REINVENTION DATE,DATE ACQUIRED,ENHANCEMENT DUE DATE,ENHANCEMENTS DUE DATE,EQUIPMENT,LEASE TERM,LINK_KEY,LOCATION,...,MAKE SURE THE OWNER SIGNS A NON-COMPETE CLAUSE,GET ITEMIZED EQUIPMENT LIST WITH CURRENT VALUE AND REPLACEMENT VALUE,NEGOTIATE WHO WILL PAY THE TRANSFER FEE,HOW LONG HAS THE CURRENT OWNER OWNED THE GYM?,HOW LONG HAS THE CURRENT EMPLOYEES BEEN AT THE GYM?,HOW HAS MEMBERSHIP RATES CHANGED OVER THE LAST 2 OR 3 YEARS,CLUB OWNER NAME,CLUB OWNER EMAIL ADDRESS,LINK,FULL ADDRESS
0,534.0,Buyer will be required to sign a service agree...,"November 1, 2022 \ 30-Day fix-it items see enh...",2019-09-16,NaN,NaN,Life Fitness,Expires 2025,/display/AFCR/4212+Coal+Ave+SE%2C+Albuquerque%...,"Albuquerque, NM",...,,,,,,,,,https://content.anytimefitness.com/display/AFC...,"4212 Coal Ave SE, Albuquerque, NM"
1,506.0,Buyer will be required to sign a service agree...,"June1, 2020",2019-09-16,NaN,NaN,Life Fitness,Expires March 2021,/display/AFCR/1108+N+Greenville+Ave%2C+Allen%2...,"Allen, TX",...,,,,,,,,,https://content.anytimefitness.com/display/AFC...,"1108 N Greenville Ave, Allen, TX"
2,288.0,Buyer will be required to sign a service agree...,1 year from the date of purchase \ 30-Day fix-...,2019-09-16,NaN,NaN,Life Fitness,"Expires October 31, 2022",/display/AFCR/2240+Prairie+Ave%2C+Beloit%2C+WI,"Beloit, WI",...,,,,,,,,,https://content.anytimefitness.com/display/AFC...,"2240 Prairie Ave, Beloit, WI"
3,333.0,Buyer will be required to sign a service agree...,"April 1, 2022 \ 30-day fix-it item see enhance...",2019-09-16,NaN,NaN,Life Fitness,Expires October 2020,/display/AFCR/1984+Veterans+Memorial+Dr%2C+Bir...,"Birmingham, AL",...,,,,,,,,,https://content.anytimefitness.com/display/AFC...,"1984 Veterans Memorial Dr, Birmingham, AL"
4,542.0,Buyer will be required to sign a service agree...,1 year from the date of purchase \ 30-day fix-...,2019-09-16,NaN,NaN,Star Trac,"Expires March 31, 2023",/display/AFCR/3177+Linden+Dr%2C+Bristol%2C+VA,"Bristol, VA",...,,,,,,,,,https://content.anytimefitness.com/display/AFC...,"3177 Linden Dr, Bristol, VA"


In [9]:
listings_filename = 'listings_' + datetime.now().strftime("%Y-%m-%d_%H%M") + '.xlsx'
#dfListings.to_excel('C:\\Users\\dwlan\\OneDrive\\The Landry Group\\Anytime Fitness\\Clubs for Sale\\' + listings_filename)
dfListings.to_excel('E:\\OneDrive\\The Landry Group\\Anytime Fitness\\Clubs for Sale\\' + listings_filename)
#dfListings.to_excel('N:\\users\\dwlandry\\OneDrive\\The Landry Group\\Anytime Fitness\\Clubs for Sale\\' + listings_filename)
#dfListings.to_excel('./' + listings_filename)
#dfListings.to_excel('./listings.xlsx')

number_of_listings = dfListings.shape[0]-1
#send_sms(listings_filename, number_of_listings)

In [15]:
dfRecord.to_excel('./listings2.xlsx')